In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('07.2022---05.2024.xlsx')
print(df['category'].unique())
df

['Restuarant' 'Market' 'Coffe' 'Transport' 'Other' 'Phone' 'Communal'
 'Clothing' 'Motel' 'Travel' 'Rent Car' 'Sport' 'Events' 'Learning'
 'Health' 'Taxi' 'Business lunch' 'Film/enjoyment' 'Tech' 'joy' 'Fuel']


,date,time,category,amount
0,2022-07-06,05:57:10,Restuarant,5.50
1,2022-07-06,05:57:27,Market,2.00
2,2022-07-06,05:58:12,Coffe,30.10
3,2022-07-06,05:58:25,Market,17.33
4,2022-07-06,05:59:00,Restuarant,5.50
...,...,...,...,...
2774,2024-04-30,05:49:01,Restuarant,8.26
2775,2024-04-30,19:58:00,Restuarant,63.00
2776,2024-05-01,06:29:58,Restuarant,8.26
2777,2024-05-01,12:38:25,Restuarant,14.00


In [3]:
import pandas as pd

# Read the data
df = pd.read_excel('07.2022---05.2024.xlsx')

# Combine 'date' and 'time' columns into a single datetime column
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))

# Drop 'date' and 'time' columns
df = df.drop(['date', 'time'], axis=1)

# Encode categorical variable 'category' using one-hot encoding
df = pd.get_dummies(df, columns=['category'])

# Set 'datetime' column as index
df.set_index('datetime', inplace=True)

# Sort DataFrame based on index
df.sort_index(inplace=True)

# Display the DataFrame after setting index and sorting
df.head(3)

,amount,category_Business lunch,category_Clothing,category_Coffe,category_Communal,category_Events,category_Film/enjoyment,category_Fuel,category_Health,category_Learning,...,category_Other,category_Phone,category_Rent Car,category_Restuarant,category_Sport,category_Taxi,category_Tech,category_Transport,category_Travel,category_joy
datetime,,,,,,,,,,,,,,,,,,,,,
2022-07-06 05:57:10,5.5,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2022-07-06 05:57:27,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-07-06 05:58:12,30.1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Read the data
df = pd.read_excel('07.2022---05.2024.xlsx')

# Combine 'date' and 'time' columns into a single datetime column
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))

# Drop 'date' and 'time' columns
df = df.drop(['date', 'time'], axis=1)

# Encode categorical variable 'category' using one-hot encoding
df = pd.get_dummies(df, columns=['category'])

# Set 'datetime' column as index
df.set_index('datetime', inplace=True)

# Create a DateTimeIndex with frequency 'D' (daily)
new_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

# Create a new DataFrame with the new index
new_df = pd.DataFrame(index=new_index)

# Merge the new DataFrame with the existing DataFrame
df = new_df.join(df)

# Convert all columns to numeric data type
df = df.apply(pd.to_numeric, errors='ignore')

# Define SARIMA parameters (p, d, q) and (P, D, Q, S)
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)

# Train SARIMA model for each category separately
sarima_models = {}
for category in df.columns[1:]:
    model = SARIMAX(df[category], order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    sarima_models[category] = model.fit()

# Forecast using SARIMA model
sarima_forecasts = {}
for category, model in sarima_models.items():
    forecast = model.predict(start=df.index[-1], end=df.index[-1] + pd.DateOffset(months=1), dynamic=True)
    sarima_forecasts[category] = forecast

# Display forecasts for the next month
sarima_forecasts_df = pd.DataFrame(sarima_forecasts)
print("SARIMA Forecasts for the Next Month:")
print(sarima_forecasts_df.tail(3))

# Calculate monthly estimates for each category as the sum of forecasted values
monthly_sum = sarima_forecasts_df.resample('M').sum()
print("\nMonthly Estimates for Each Category (Sum of Forecasted Values):")
monthly_sum

c:\ProgramData\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\ProgramData\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c

SARIMA Forecasts for the Next Month:
                     category_Business lunch  category_Clothing  \
2024-05-30 05:57:10                      0.0                0.0   
2024-05-31 05:57:10                      0.0                0.0   
2024-06-01 05:57:10                      0.0                0.0   

                     category_Coffe  category_Communal  category_Events  \
2024-05-30 05:57:10             0.0                0.0              0.0   
2024-05-31 05:57:10             0.0                0.0              0.0   
2024-06-01 05:57:10             0.0                0.0              0.0   

                     category_Film/enjoyment  category_Fuel  category_Health  \
2024-05-30 05:57:10                      0.0            0.0              0.0   
2024-05-31 05:57:10                      0.0            0.0              0.0   
2024-06-01 05:57:10                      0.0            0.0              0.0   

                     category_Learning  category_Market  ...  category_O

,category_Business lunch,category_Clothing,category_Coffe,category_Communal,category_Events,category_Film/enjoyment,category_Fuel,category_Health,category_Learning,category_Market,...,category_Other,category_Phone,category_Rent Car,category_Restuarant,category_Sport,category_Taxi,category_Tech,category_Transport,category_Travel,category_joy
2024-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1191.666667,0.0,0.0,0.0,0.0,0.0,0.0
2024-06-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,39.666667,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

def read_data(file_path):
    # Read the data from the specified Excel file
    df = pd.read_excel(file_path)
    return df

def preprocess_data(df):
    # Combine 'date' and 'time' columns into a single datetime column
    df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), errors='coerce')
    
    # Drop 'date' and 'time' columns
    df = df.drop(['date', 'time'], axis=1)
    
    # Sort the dataframe by the datetime index
    df = df.sort_values(by='datetime')
    
    # Set 'datetime' column as index
    df.set_index('datetime', inplace=True)
    
    # Set the frequency of the datetime index
    df.index.freq = pd.infer_freq(df.index)
    
    return df

def train_sarima_model(df, order, seasonal_order):
    # Train SARIMA model for each category separately
    sarima_models = {}
    for category in df.columns[1:]:
        model = SARIMAX(df[category], order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        
        # Explicitly convert the endogenous variable to a numpy array
        endog = np.asarray(df[category])
        
        # Fit the model using the numpy array
        sarima_models[category] = model.fit(start_params=np.random.random(size=model.k_params), method='powell', maxiter=1000, disp=False, start_ar_lags=order[0])
    return sarima_models

def forecast_next_month(sarima_models, df):
    # Forecast using SARIMA model for the next month
    sarima_forecasts = {}
    for category, model in sarima_models.items():
        forecast = model.forecast(steps=1)
        sarima_forecasts[category] = forecast.iloc[0]  # Access the first forecasted value
    # Create DataFrame for forecasts
    sarima_forecasts_df = pd.DataFrame.from_dict(sarima_forecasts, orient='index', columns=['Forecast'])
    sarima_forecasts_df.index.name = 'Category'  # Set index name to 'Category'
    print(sarima_forecasts_df)  # Print DataFrame before converting index to datetime
    sarima_forecasts_df.index = pd.to_datetime(sarima_forecasts_df.index, errors='coerce')  # Convert index to datetime
    sarima_forecasts_df = sarima_forecasts_df.dropna()  # Drop rows with NaT values
    return sarima_forecasts_df

def get_monthly_estimates(sarima_forecasts_df):
    # Drop rows with NaT values in the index
    sarima_forecasts_df = sarima_forecasts_df.dropna()
    
    if not sarima_forecasts_df.empty:
        # Check if index contains datetime values
        if isinstance(sarima_forecasts_df.index, pd.DatetimeIndex):
            # Print DataFrame before resampling
            print("DataFrame before resampling:")
            print(sarima_forecasts_df)
            
            # Calculate monthly estimates for each category as the sum of forecasted values
            monthly_sum = sarima_forecasts_df.resample('M').sum()
            return monthly_sum
        else:
            print("Index does not contain datetime values. Cannot resample.")
            return None
    else:
        print("No valid data to calculate monthly estimates.")
        return None




def get_forecasts_for_month(month, sarima_forecasts_df):
    # Filter forecasts for the given month
    forecasts_for_month = sarima_forecasts_df.loc[month]
    
    # Convert the forecasts to a dictionary
    forecasts_dict = forecasts_for_month.to_dict()
    
    # Remove the 'category_' prefix from the keys
    forecasts_dict = {category.split('_')[1]: value for category, value in forecasts_dict.items()}
    
    return forecasts_dict

if __name__ == "__main__":
    # Define SARIMA parameters (p, d, q) and (P, D, Q, S)
    order = (1, 1, 1)
    seasonal_order = (1, 1, 1, 12)
    
    # Read the data
    df = read_data('07.2022---05.2024.xlsx')
    
    # Preprocess the data
    df = preprocess_data(df)
    
    # Train SARIMA model
    sarima_models = train_sarima_model(df, order, seasonal_order)
    
    # Forecast for the next month
    sarima_forecasts_df = forecast_next_month(sarima_models, df)
    
    # Display forecasts for the next month
    print("SARIMA Forecasts for the Next Month:")
    print(sarima_forecasts_df.tail(3))
    
    # Calculate monthly estimates
    monthly_sum = get_monthly_estimates(sarima_forecasts_df)
    print("\nMonthly Estimates for Each Category (Sum of Forecasted Values):")
    print(monthly_sum)
    
    # Example usage: Get forecasts for a specific month
    month = '2024-05'  # Specify the month for which you want to get forecasts
    forecasts_for_month = get_forecasts_for_month(month, sarima_forecasts_df)
    
    # Print the forecasts for the specified month
    for category, value in forecasts_for_month.items():
        print(f"{category}: {value}")


c:\ProgramData\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\lib\site-packages\statsmodels\base\optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method powell is: xtol, ftol, maxfun, start_direc. The list of unsupported keyword arguments passed include: start_ar_lags. After release 0.14, this will raise.
  warnings.warn(


          Forecast
Category          
amount   -1.610746
SARIMA Forecasts for the Next Month:
          Forecast
Category          
NaT      -1.610746
DataFrame before resampling:
          Forecast
Category          
NaT      -1.610746


c:\ProgramData\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


AttributeError: 'NaTType' object has no attribute 'normalize'

In [ ]:
# from fbprophet import Prophet

# # Train Prophet model for each category separately
# prophet_models = {}
# for category in df.columns[1:]:
#     df_prophet = df[[category]].reset_index().rename(columns={'datetime':'ds', category:'y'})
#     model = Prophet()
#     model.fit(df_prophet)
#     prophet_models[category] = model

# # Forecast using Prophet model
# prophet_forecasts = {}
# for category, model in prophet_models.items():
#     future = model.make_future_dataframe(periods=30, freq='D')
#     forecast = model.predict(future)
#     prophet_forecasts[category] = forecast[['ds', 'yhat']].set_index('ds').rename(columns={'yhat':category})


In [6]:
# from sklearn.linear_model import LinearRegression

# # Train Linear Regression model for each category separately
# linear_models = {}
# for category in df.columns[1:]:
#     X = df.index.values.reshape(-1, 1)
#     y = df[category]
#     model = LinearRegression()
#     model.fit(X, y)
#     linear_models[category] = model

# # Forecast using Linear Regression model
# linear_forecasts = {}
# for category, model in linear_models.items():
#     start_date = df.index[-1]
#     end_date = start_date + pd.DateOffset(months=1)
#     future_dates = pd.date_range(start=start_date, end=end_date, freq='D')[1:]  # Exclude start_date
#     X_future = future_dates.values.reshape(-1, 1)
#     forecast = model.predict(X_future)
#     forecast_index = pd.Index(future_dates, name='datetime')
#     linear_forecasts[category] = pd.Series(forecast, index=forecast_index, name=category)
